In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import pandas as pd
import time

product_name = "[99시리즈] 인기 야식 6종 (택1)"
product_url = "https://www.kurly.com/goods/1001146322"
output_csv_name = "kurly_reviews_final.csv"
max_reviews = 1000  # 수집 최대 리뷰 수

options = Options()
options.add_argument('--start-maximized')
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)

print(f"[INFO] '{product_name}' 리뷰 수집 시작")
driver.get(product_url)
time.sleep(3)

try:
    review_tab = driver.find_element(By.XPATH, '//a[.//span[text()="후기"]]')
    driver.execute_script("arguments[0].scrollIntoView(true);", review_tab)
    time.sleep(1.5)
    review_tab.click()
    time.sleep(3)
except:
    print("후기 탭 클릭 생략됨")

review_page = 1
product_reviews = []

while True:
    time.sleep(3)

    review_blocks = driver.find_elements(By.CSS_SELECTOR, "article.css-inxa61")
    grade_blocks = driver.find_elements(By.CSS_SELECTOR, "div.css-itusxp")

    if len(review_blocks) == 0:
        print("리뷰 없음 → 종료")
        break

    print(f"📄 리뷰 페이지 {review_page} ({len(review_blocks)}개 수집)")

    for idx, block in enumerate(review_blocks):
        if len(product_reviews) >= max_reviews:
            print(f"\n 최대 리뷰 수({max_reviews}) 도달 → 수집 종료")
            driver.quit()
            df = pd.DataFrame(product_reviews)
            df.to_csv(output_csv_name, index=False, encoding='utf-8-sig', escapechar='\\')
            print(f"\n 총 {len(df)}개 리뷰 저장 완료 → {output_csv_name}")
            exit()

        try:
            text = block.find_element(By.CSS_SELECTOR, "p.css-y49dcn").text.strip()
        except:
            text = ""

        try:
            date = block.find_element(By.CSS_SELECTOR, "span.css-14kcwq8").text.strip()
        except:
            date = ""

        try:
            has_img = "Y" if block.find_element(By.CSS_SELECTOR, "img[alt='리뷰 이미지 썸네일']") else "N"
        except:
            has_img = "N"

        try:
            help_btn = block.find_element(By.CSS_SELECTOR, 'button.css-g3a39p')
            help_count = ''.join(filter(str.isdigit, help_btn.text.strip())) or '0'
        except:
            help_count = '0'

        try:
            grade_block = grade_blocks[idx]
            grade_elements = grade_block.find_elements(By.CSS_SELECTOR, "div[class*='6y4urb']")
            grade = ' '.join([g.text.strip() for g in grade_elements])
        except:
            grade = ""

        try:
            option_block = block.find_element(By.CSS_SELECTOR, "div.css-18pn4xv")
            option = option_block.find_element(By.TAG_NAME, "h3").text.strip()
        except:
            option = ""

        product_reviews.append({
            "상품명": product_name,
            "리뷰내용": text,
            "작성일": date,
            "이미지포함": has_img,
            "도움돼요수": int(help_count),
            "회원등급": grade,
            "상품옵션": option,
            "상품URL": product_url
        })

    # 다음 페이지 이동
    try:
        time.sleep(2.5)
        next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.css-1orps7k.ebs5rpx0")))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_btn)
        time.sleep(2.5)
        next_btn.click()
        review_page += 1
    except StaleElementReferenceException:
        print("StaleElement → 재탐색")
        time.sleep(2)
        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, "button.css-1orps7k.ebs5rpx0")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_btn)
            time.sleep(2.5)
            next_btn.click()
            review_page += 1
        except:
            print("재시도 실패 → 종료")
            break
    except TimeoutException:
        print(" 더 이상 다음 페이지 없음 → 종료")
        break
    except Exception as e:
        print(f" 다음 버튼 클릭 실패: {e}")
        break

driver.quit()

df = pd.DataFrame(product_reviews)
df.to_csv(output_csv_name, index=False, encoding='utf-8-sig', escapechar='\\')
print(f"\n 총 {len(df)}개 리뷰 저장 완료 → {output_csv_name}")
